**Dinámica Molecular del Agua usando TIP4P en OpenMM**

**1. Instalar dependencias**

In [ ]:
# Install conda (via mamba)
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install OpenMM and OpenMMTools using mamba
!mamba install -c conda-forge openmm openmmtools --quiet

In [ ]:
!pip install --quiet nglview parmed mdanalysis

**2. Importar librerías necesarias**

In [ ]:
# OpenMM Libraries
import openmm
from openmm import unit, LangevinIntegrator, Platform, MonteCarloBarostat, XmlSerializer
from openmm.app import Simulation, PDBReporter, StateDataReporter, ForceField, PME, HBonds, DCDReporter, PDBFile

# OpenMMTools Libraries
from openmmtools.testsystems import WaterBox

# Analysis and Visualization Libraries
import numpy as np
import matplotlib.pyplot as plt
from sys import stdout

# Additional Libraries (if needed, for file operations and time)
import time
import os

# MDAnalysis libraries
import MDAnalysis as mda
from MDAnalysis.analysis import rdf
#from MDAnalysis.coordinates.PDB import PDBWrite

# **Inicialización y Definición del sistema**

**3. Crear una caja de agua (TIP4P)**

TIP4P es un modelo rígido de agua de 4 sitios, usado frecuentemente por su precisión

In [ ]:
# Crear una caja de agua TIP4P-Ew
waterbox = WaterBox(model='tip4pew', box_edge=3.0*unit.nanometers)

# El sistema y las posiciones iniciales están fácilmente disponibles.
system = waterbox.system
positions = waterbox.positions


# **Explicación:**

Esta línea crea una caja cúbica llena de moléculas de agua, usando la clase WaterBox de la librería **openmmtools.testsystems**. Este objeto waterbox contiene:

* El sistema molecular (waterbox.system)
* Las posiciones iniciales de todos los átomos (waterbox.positions)
* La topología del sistema

**WaterBox()**
*Es una clase que genera automáticamente un sistema con moléculas de agua en una caja cúbica. Es útil para hacer simulaciones rápidas de equilibrio de agua o como base para otros sistemas más complejos.*

**model='tip4pew'**
*Este argumento indica qué modelo de agua usar.*

¿Qué es un campo de fuerza (force field)?
*Un campo de fuerza define las reglas físicas del sistema:*

* Cómo se comportan los enlaces
* Cómo interactúan los átomos (fuerzas, potenciales)
* Parámetros como cargas, radios, etc.

¿Por qué TIP4P-Ew?

*Es una versión del modelo TIP4P diseñada para funcionar bien con electrostática tratada con el método Ewald (PME). Existen otros modelos como 'tip3p', 'tip4p', 'spce', etc.*

*Representa el agua con 4 sitios: 3 átomos reales (O, H, H) y un cuarto punto de carga negativa que mejora el momento dipolar.*

*Tiene mejor precisión en propiedades físicas como densidad, punto de fusión, y comportamiento dieléctrico en comparación con modelos más simples como SPC o TIP3P.*

**box_edge=3.0*unit.nanometers**
*Este argumento define el tamaño del lado de la caja cúbica.*

En este caso, estás creando una caja de 3.0 × 3.0 × 3.0 nanómetros cúbicos.

La cantidad de moléculas de agua que se agregan se calcula automáticamente para que la densidad sea realista (~1 g/cm³).


Despues se extraen los componentes del objeto waterbox:

**system = waterbox.system**
*Es el objeto System de OpenMM que contiene:*

* Las masas, restricciones y conexiones de las moléculas
* Las fuerzas definidas por el campo de fuerza (interacciones de Van der Waals, electrostáticas, etc.)
* Información necesaria para realizar la simulación

**positions = waterbox.positions**
*Son las posiciones iniciales de todos los átomos en el sistema. Las moléculas se colocan en posiciones razonables que simulan agua líquida.*


# **Visualización**

**4. Guardar PDB temporalmente para visualizar**

In [ ]:
# Guarda el archivo PDB a disco
with open('waterbox.pdb', 'w') as f:
    PDBFile.writeFile(waterbox.topology, waterbox.positions, f)

# Cargar el sistema desde el PDB generado por OpenMM
u = mda.Universe("waterbox.pdb")

# Envolver las moléculas en la celda
u.atoms.wrap(inplace=True)

# Guardar el sistema envuelto
with mda.Writer("waterbox_wrapped.pdb", multiframe=False) as w:
    w.write(u.atoms)



# **Minimización de energía**

**5. Minimización de energía**

La minimización elimina contactos atómicos malos o solapamientos

In [ ]:
# Definir un integrador
temperature = 300 * unit.kelvin
friction = 1 / unit.picosecond
step_size = 0.002 * unit.picoseconds
integrator = LangevinIntegrator(temperature, friction, step_size)

# Propiedades de la plataforma
properties = {}
platforms = [Platform.getPlatform(i) for i in range(Platform.getNumPlatforms())]
platform_names = [platform.getName() for platform in platforms]
platform = Platform.getPlatformByName('CUDA') if 'CUDA' in platform_names else Platform.getPlatformByName('Reference')

simulation = Simulation(waterbox.topology, system, integrator, platform, properties)

# Establecer las posiciones iniciales
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature)

In [ ]:
# Minimizar la energía (~11min)
print("Minimizando la energía...")
simulation.minimizeEnergy()
print("Minimización de la energía terminada.")


In [ ]:
# Guardar el estado del sistema después de la minimización
with open('minimized_state.xml', 'w') as f:
    f.write(XmlSerializer.serialize(simulation.context.getState(getPositions=True, getVelocities=True)))

# **Dinámica Molecular (MD)**

**6. Equilibración NVT (Temperatura constante)**

Primero relajamos la temperatura con volumen fijo. T = 300 K. Time elapsed ~40 min

In [ ]:
# Establecer velocidades para la equilibración NVT
n_steps_nvt = 5000

# Cargar el estado minimizado
with open('minimized_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

# Crear un nuevo Simulation object si es necesario
#simulation = Simulation(topology, system, integrator, platform)
simulation.context.setState(state)

# Equilibración NVT
print("\nEjecutando la equilibración NVT...")
simulation.reporters.append(StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True, density=True, progress=True, totalSteps=n_steps_nvt))

# Ejecutar la equilibración NVT
simulation.step(n_steps_nvt)

In [ ]:
# Guardar estado después de NVT
with open('nvt_state.xml', 'w') as f:
    f.write(XmlSerializer.serialize(simulation.context.getState(getPositions=True, getVelocities=True)))

**7. Equilibración NPT (Temperatura y presión constantes)**

Ahora dejamos que el sistema relaje su densidad. T = 300K, P = 1 atm. ~1 hr 20 min

In [ ]:
# Leer el estado después de NVT
with open('nvt_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

# Preparar sistema NPT
forcefield = ForceField('amber14-all.xml', 'tip4pew.xml')
modeller = waterbox  # debe ser el mismo que usaste antes
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1.0 * unit.nanometer,
                                 constraints=HBonds)
barostat = MonteCarloBarostat(1 * unit.atmosphere, 300 * unit.kelvin)
system.addForce(barostat)

integrator = LangevinIntegrator(300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds)
simulation = Simulation(modeller.topology, system, integrator, platform, properties)

# Aplicar el estado guardado
simulation.context.setState(state)

# Limpia y añade reporteros
simulation.reporters.clear()
n_steps_print = 1000
n_steps_npt = 10000
simulation.reporters.append(DCDReporter('NPTequilibration_trajectory.dcd', n_steps_print))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True, totalEnergy=True, temperature=True, density=True, progress=True, totalSteps=n_steps_npt))
simulation.reporters.append(StateDataReporter('NPTequilibration_data.csv', 100, step=True, totalEnergy=True, temperature=True, density=True, volume=True))

# Correr NPT
print("\nEjecutando la equilibración NPT...")
start = time.time()
simulation.step(n_steps_npt)
end = time.time()
print(f"Tiempo transcurrido: {end-start:.2f} segundos")

**Guardar sistema equilibrado para reiniciar la simulacion despues**

In [ ]:
# Guardar estado al final de la equilibración
with open('equilibrated_state.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())

**Analisis de propiedades termodinámicas del sistema**

In [ ]:
# Leer los datos del archivo CSV para graficar
data = np.genfromtxt('NPTequilibration_data.csv', delimiter=',', names=True)
print(data.dtype.names)

steps = data['Step']

# Graficar Energía
plt.figure(figsize=(6,3))
plt.plot(steps, data['Total_Energy_kJmole'], color='black', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Total Energy (kJ/mol)', fontsize=13)
plt.ylim(-37000, 0)
plt.grid(True)
#plt.savefig('energy_plot.png')

# Graficar Volumen
plt.figure(figsize=(6,3))
plt.plot(steps, data['Box_Volume_nm3'], color='lightskyblue', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Volume (units)', fontsize=13)
plt.ylim(0,30)
plt.grid(True)
#plt.savefig('volume_plot.png')

# Graficar Temperatura
plt.figure(figsize=(6,3))
plt.plot(steps, data['Temperature_K'], color='lightgreen', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Temperature (K)', fontsize=13)
plt.ylim(0,350)
plt.grid(True)
#plt.savefig('temperature_plot.png')

# Graficar Densidad
plt.figure(figsize=(6,3))
plt.plot(steps, data['Density_gmL'], color='hotpink', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Density (g/mL)', fontsize=13)
plt.ylim(0,1.2)
plt.grid(True)
#plt.savefig('density_plot.png')

# **Explicación:**

**¿Qué hacen las simulaciones NVT y NPT?**

*NVT (ensemble canónico):*

* Número de partículas (N), volumen (V) y temperatura (T) se mantienen constantes.
* Se usa para relajar el sistema a una temperatura deseada, sin cambiar el tamaño de la caja.
* Aquí el volumen está fijo → la densidad no cambia.

*NPT (ensemble isóbarico-isotérmico):*

* Número de partículas (N), presión (P) y temperatura (T) se mantienen constantes.
* Se usa para relajar el sistema a la presión deseada, permitiendo que la caja cambie de volumen → la densidad se ajusta.
* Ideal para condiciones más realistas (como 1 atm, 300 K).

**¿Por qué primero NVT y luego NPT?**
1. *Control progresivo del sistema*
Después de minimizar la energía, el sistema puede estar en un estado artificial:
* Temperatura ≈ 0 K (porque no hay velocidades).
* Geometría realista pero sin movimiento.

**Primero usamos NVT:**
* Para asignar velocidades y relajar las moléculas a la temperatura objetivo sin cambios en el volumen.
* Si usaras NPT directamente, el sistema podría cambiar su volumen de forma inestable debido a fluctuaciones grandes de energía/temperatura.

2. *Evitar colapsos o explosiones del sistema*
**Ir directamente a NPT sin una fase de NVT puede causar:**
* Cambios drásticos en volumen → densidad irreal.
* Problemas de convergencia → átomos muy juntos o muy separados.

3. *Preparar condiciones para simulaciones de producción*
* Después del NVT, el sistema tiene la temperatura deseada.
* Después del NPT, el sistema tiene la densidad y volumen correctos (equilibrados a la presión deseada).
* Ahora puedes correr simulaciones de producción con confianza de que estás en condiciones realistas.

# **Corrida de producción**

**10. Corrida de producción (NPT)**

Ahora hacemos la simulación principal

In [ ]:
# Cargar checkpoint
with open('equilibrated_state.chk', 'rb') as f:
    simulation.context.loadCheckpoint(f.read())

# Agregar reporteros para la corrida de producción
simulation.reporters.clear()

# Archivos de salida
simulation.reporters.append(DCDReporter('production_trajectory.dcd', 2000))
simulation.reporters.append(StateDataReporter(stdout, 200, step=True, totalEnergy=True, temperature=True, density=True, progress=True, totalSteps=20000))
simulation.reporters.append(StateDataReporter('production_data.csv', 200, step=True, totalEnergy=True, temperature=True, density=True))

# Ejecutar pasos de producción
print("\nIniciando simulación de producción NPT (20000 pasos)...")
start = time.time()
simulation.step(20000)
end = time.time()
print(f"Simulación de producción completada en {end - start:.2f} segundos.")

In [ ]:
# Guardar estado como checkpoint binario
with open('final_production_state.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())

# Guardar posiciones finales como archivo .pdb
state = simulation.context.getState(getPositions=True)
with open('final_production_structure.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, state.getPositions(), f)

# Guardar estado completo (opcional, pero útil para reproducibilidad)
with open('final_production_state.xml', 'w') as f:
    f.write(XmlSerializer.serialize(state))

print("Archivos guardados:")
print("- final_production_state.chk (para continuar simulación)")
print("- final_production_structure.pdb (para análisis/visualización)")
print("- final_production_state.xml (opcional, estado legible)")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Leer los datos del archivo CSV para graficar
csv_filename = 'production_data.csv'
data = np.genfromtxt(csv_filename, delimiter=',', names=True)
print(data.dtype.names)

steps = data['Step']

# Graficar Energía
plt.figure(figsize=(6,3))
plt.plot(steps, data['Total_Energy_kJmole'], color='black', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Energy (kJ/mol)', fontsize=13)
plt.ylim(-37000, 0)
plt.grid(True)
#plt.savefig('energy_plot.png')

# Graficar Temperatura
plt.figure(figsize=(6,3))
plt.plot(steps, data['Temperature_K'], color='yellow', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Temperature (K)', fontsize=13)
plt.ylim(0,350)
plt.grid(True)
#plt.savefig('temperature_plot.png')

# Graficar Densidad
plt.figure(figsize=(6,3))
plt.plot(steps, data['Density_gmL'], color='pink', linewidth=3)
plt.xlabel('Step', fontsize=13)
plt.ylabel('Density (g/mL)', fontsize=13)
plt.ylim(0,1.2)
plt.grid(True)
#plt.savefig('density_plot.png')